# Projeto 1 - Ciência dos Dados

Nome: _____

Nome: _____

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\pedro\Documents\GitHub\CDADOS-PROJETO1


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
filename = 'Burger King.xlsx'

In [4]:
train = pd.read_excel(filename)
train.head(30)

,Treinamento,B
0,queria um burger king agora https://t.co/ddr6r...,1
1,as vezes publicitários me irritam tanto q tive...,1
2,ano que vem burger king vai ta patrocinando a ...,0
3,"só um burger king, é sobre isso",0
4,minha fome tem nome específico burger king o ...,1
5,eu odeio o marketing do burger king\nmas eu od...,1
6,burger king sai da economia por você espero mi...,0
7,que saudade de ir no burger king,1
8,gostaria de propor uma rinha: burger king ou m...,0
9,off todas as vzs que peco no bk meu hamburguer...,1


In [5]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,B
0,rt @_bradachi: mc donalds é melhor que burger ...,1
1,mds eu acabei de ver uma propagando no yt do b...,1
2,@burgerkingbr todas as vezes que fui no burger...,1
3,@souamorgan eu te entendo... tipo logo abaixo ...,1
4,ifood e burger king sempre deixa forte kkk 🙅💪😅,0


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

*O nosso produto é a marca Burger King.Consideramos relevante tudo aquilo que falava positivamente ou negativamente sobre nosso produto.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [6]:
#LIMPADOR DE PLAVRAS V.10000000
import re 
i=0
def cleanup(text):

    punctuation = '[!-.:?;/,""''@#$%^&*()+=]'
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

Train = train.copy()
while i < len(train):
    Train.loc[i,'Treinamento'] = cleanup(Train.loc[i,'Treinamento'])
    i+= 1
    
#A pd.Series Train contem a tabela train porem sem os caracteres 

#Criando uma serie com as palavras relevantes e nao relevantes e fazer tabela de frequencia delas
palavras = Train.loc[Train['B'] == 1]
palavras = palavras['Treinamento'].tolist()
palavras = " ".join(palavras)
palavras = palavras.split()
serie_palavras_relevantes = pd.Series(palavras) #serie_palavras_relevantes

irrel = Train.loc[Train['B'] == 0]
irrel = irrel['Treinamento'].tolist()
irrel = " ".join(irrel)
irrel = irrel.split()
serie_palavras_irrelevantes = pd.Series(irrel) #serie_palavras_irrelevantes

tabela_relevantes = serie_palavras_relevantes.value_counts() #quantas vezes cada palavra relevante aparece
tabela_irrelevantes = serie_palavras_irrelevantes.value_counts() #quantas vezes cada palavra irrelevante aparece
tabela_relevantes_relativa = serie_palavras_relevantes.value_counts(sort=True,normalize=True) #frequencia relativa de todas as palavras relevantes
tabela_irrelevantes_relativa = serie_palavras_irrelevantes.value_counts(sort=True,normalize=True) #frequencia relativa de todas as palavras irrelevantes

portugues = palavras + irrel #todas as palavras
serie_portugues = pd.Series(portugues)
tabela_portugues = serie_portugues.value_counts() #quantas vezes cada palavra aparece na planilha
tabela_portugues_relativa = serie_portugues.value_counts(sort=True,normalize=True)

In [7]:
tabela_irrelevantes_relativa['burger']

0.057594747836466724

In [15]:
#fazendo o classificador

resultados = []

def classificador(frase):
        frase = cleanup(frase)
        probR = tabela_relevantes.sum()/(tabela_irrelevantes.sum()+tabela_relevantes.sum())
        probI = tabela_irrelevantes.sum()/(tabela_irrelevantes.sum()+tabela_relevantes.sum())
        P_frase_relevante = 1
        P_frase_irrelevante= 1
        for palavra in frase.split():
            if palavra in tabela_relevantes:
                
                Counts_Palavra_R = tabela_relevantes[palavra]
                
            else:
                Counts_Palavra_R = 0

            P_Palavra_R = (Counts_Palavra_R + 1) / (tabela_relevantes.sum() + tabela_portugues.sum())
            
            P_frase_relevante *= P_Palavra_R
            
            if palavra in tabela_irrelevantes:
                Counts_Palavra_I = tabela_irrelevantes[palavra]
                
            else:
                Counts_Palavra_I = 0
            
            P_Palavra_I = (Counts_Palavra_I + 1) / (tabela_irrelevantes.sum() + tabela_portugues.sum())
            P_frase_irrelevante *= P_Palavra_I
            

        P_R_dado_frase = P_frase_relevante*probR
        P_I_dado_frase = P_frase_irrelevante*probI
        
        if P_R_dado_frase > P_I_dado_frase:

            return 1
        else :
            
            return 0
        
        

0      0
1      0
2      1
3      0
4      0
      ..
147    0
148    0
149    1
150    0
151    0
Name: resultados, Length: 152, dtype: int64

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [35]:

for frase in test['Teste']:
    resultado = classificador(frase)
    resultados.append(resultado)
        
test['resultados'] = pd.Series(resultados)


verd_negativos = pd.crosstab(test['resultados'], test['B'],normalize=True)[0][0]
verd_positivos = pd.crosstab(test['resultados'], test['B'],normalize=True)[1][1]
falso_negativos = pd.crosstab(test['resultados'], test['B'],normalize=True)[1][0]
falso_positivo = pd.crosstab(test['resultados'], test['B'],normalize=True)[0][1]
print("porcentagem verdadeiros negativos:{0}\n".format(verd_negativos))
print("porcentagem verdadeiros positivos:{0}\n".format(verd_positivos))
print("porcentagem falsos negativos:{0}\n".format(falso_negativos))
print("porcentagem falsos positivos:{0}\n".format(falso_positivo))
pd.crosstab(test['resultados'], test['B'],normalize=True)

porcentagem verdadeiros negativos:0.45394736842105265

porcentagem verdadeiros positivos:0.06578947368421052

porcentagem falsos negativos:0.39473684210526316

porcentagem falsos positivos:0.08552631578947369



B,0,1
resultados,,
0,0.453947,0.394737
1,0.085526,0.065789


___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)


___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**